In [1]:
pip install cfbd --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cfbd
import re
from datetime import datetime

In [3]:
with open('keys/keys.json') as fi:
    credentials = json.load(fi)

In [4]:
api_key = credentials['api_key']

In [5]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = api_key
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

In [6]:
teams_api = cfbd.TeamsApi(api_config)

In [7]:
teams_api.get_fbs_teams()

[{'abbreviation': 'AFA',
  'alt_color': '#ffffff',
  'alt_name_1': None,
  'alt_name_2': None,
  'alt_name_3': None,
  'classification': None,
  'color': '#004a7b',
  'conference': 'Mountain West',
  'division': 'Mountain',
  'id': 2005,
  'location': {'capacity': 46692.0,
               'city': 'Colorado Springs',
               'country_code': 'US',
               'dome': False,
               'elevation': 2024.875732,
               'grass': False,
               'latitude': 38.9969701,
               'longitude': -104.8436165,
               'name': 'Falcon Stadium',
               'state': 'CO',
               'timezone': 'America/Denver',
               'venue_id': 3713,
               'year_constructed': 1962.0,
               'zip': '80840'},
  'logos': ['http://a.espncdn.com/i/teamlogos/ncaa/500/2005.png',
            'http://a.espncdn.com/i/teamlogos/ncaa/500-dark/2005.png'],
  'mascot': 'Falcons',
  'school': 'Air Force',
  'twitter': '@AF_Football'},
 {'abbreviation': 'AKR'

In [8]:
betting_api = cfbd.BettingApi(api_config)
betting_2018 = betting_api.get_lines(year= 2018)

In [9]:
betting_2018_df = pd.DataFrame.from_records([g.to_dict() for g in betting_2018])
betting_2018_df.head()

id  season  week season_type                start_date home_team  \
0  401029895    2018     1     regular  2018-08-30T23:00:00.000Z  Delaware   
1  401029921    2018     1     regular  2018-08-30T23:00:00.000Z     Maine   
2  401030318    2018     1     regular  2018-08-30T23:30:00.000Z   Samford   
3  401030369    2018     4     regular  2018-09-22T17:00:00.000Z    Dayton   
4  401030538    2018     4     regular  2018-09-22T17:00:00.000Z    Bryant   

  home_conference  home_score      away_team away_conference  away_score lines  
0             CAA          19   Rhode Island             CAA          21    []  
1             CAA          35  New Hampshire             CAA           7    []  
2        Southern          66        Shorter      Gulf South           9    []  
3         Pioneer          42       Davidson         Pioneer          21    []  
4             NEC          49  Robert Morris             NEC          46    []

In [10]:
betting_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               1511 non-null   int64 
 1   season           1511 non-null   int64 
 2   week             1511 non-null   int64 
 3   season_type      1511 non-null   object
 4   start_date       1511 non-null   object
 5   home_team        1511 non-null   object
 6   home_conference  1510 non-null   object
 7   home_score       1511 non-null   int64 
 8   away_team        1511 non-null   object
 9   away_conference  1497 non-null   object
 10  away_score       1511 non-null   int64 
 11  lines            1511 non-null   object
dtypes: int64(5), object(7)
memory usage: 141.8+ KB


In [12]:
result=[]
for x in range(0,1511):
    match = re.search(r'\d{4}-\d{2}-\d{2}', betting_2018_df['start_date'][x])
    date = datetime.strptime(match.group(), '%Y-%m-%d').date()
    result.append(date)


In [13]:
betting_2018_df['date'] = result

In [14]:
# shift column 'Name' to first position 
fourth_column = betting_2018_df.pop('date') 
  
# insert column using insert(position,column_name, 
# first_column) function 
betting_2018_df.insert(4, 'date', fourth_column) 

In [15]:
betting_2018_df

id  season  week season_type        date  \
0     401029895    2018     1     regular  2018-08-30   
1     401029921    2018     1     regular  2018-08-30   
2     401030318    2018     1     regular  2018-08-30   
3     401030369    2018     4     regular  2018-09-22   
4     401030538    2018     4     regular  2018-09-22   
...         ...     ...   ...         ...         ...   
1506  401030505    2018    12     regular  2018-11-17   
1507  401030527    2018    12     regular  2018-11-17   
1508  401030496    2018    12     regular  2018-11-17   
1509  401019504    2018     6     regular  2018-10-06   
1510  401014989    2018     2     regular  2018-09-08   

                    start_date               home_team home_conference  \
0     2018-08-30T23:00:00.000Z                Delaware             CAA   
1     2018-08-30T23:00:00.000Z                   Maine             CAA   
2     2018-08-30T23:30:00.000Z                 Samford        Southern   
3     2018-09-22T17:00:00.000Z                  Dayton         Pioneer   
4     2018-09-22T17:00:00.000Z                  Bryant             NEC   
...                        ...                     ...             ...   
1506  2018-11-17T19:00:00.000Z          Delaware State            MEAC   
1507  2018-11-17T19:00:00.000Z  North Carolina Central            MEAC   
1508  2018-11-17T19:00:00.000Z             Florida A&M            MEAC   
1509  2018-10-06T19:30:00.000Z               Air Force   Mountain West   
1510  2018-09-08T23:00:00.000Z    Southern Mississippi  Conference USA   

      home_score           away_team    away_conference  away_score  \
0             19        Rhode Island                CAA          21   
1             35       New Hampshire                CAA           7   
2             66             Shorter         Gulf South           9   
3             42            Davidson            Pioneer          21   
4             49       Robert Morris                NEC          46   
...          ...                 ...                ...         ...   
1506          41  Virginia Lynchburg               None           7   
1507           0  North Carolina A&T               MEAC          45   
1508          19     Bethune-Cookman               MEAC          33   
1509          35                Navy  American Athletic           7   
1510          20    Louisiana Monroe           Sun Belt          21   

                                                  lines  
0                                                    []  
1                                                    []  
2                                                    []  
3                                                    []  
4                                                    []  
...                                                 ...  
1506                                                 []  
1507                                                 []  
1508                                                 []  
1509  [{'provider': 'Caesars', 'spread': 2.5, 'forma...  
1510  [{'provider': 'Caesars', 'spread': -5.0, 'form...  

[1511 rows x 13 columns]

try:
        betting_2022_df.append(pd.json_normalize(betting_2022_df['lines'][1]))
except IndexError:
        betting_2022_df.append('null')

In [16]:
betting_2018_df = betting_2018_df.explode('lines').reset_index()

In [17]:
betting_2018_df

index         id  season  week season_type        date  \
0         0  401029895    2018     1     regular  2018-08-30   
1         1  401029921    2018     1     regular  2018-08-30   
2         2  401030318    2018     1     regular  2018-08-30   
3         3  401030369    2018     4     regular  2018-09-22   
4         4  401030538    2018     4     regular  2018-09-22   
...     ...        ...     ...   ...         ...         ...   
3563   1509  401019504    2018     6     regular  2018-10-06   
3564   1510  401014989    2018     2     regular  2018-09-08   
3565   1510  401014989    2018     2     regular  2018-09-08   
3566   1510  401014989    2018     2     regular  2018-09-08   
3567   1510  401014989    2018     2     regular  2018-09-08   

                    start_date             home_team home_conference  \
0     2018-08-30T23:00:00.000Z              Delaware             CAA   
1     2018-08-30T23:00:00.000Z                 Maine             CAA   
2     2018-08-30T23:30:00.000Z               Samford        Southern   
3     2018-09-22T17:00:00.000Z                Dayton         Pioneer   
4     2018-09-22T17:00:00.000Z                Bryant             NEC   
...                        ...                   ...             ...   
3563  2018-10-06T19:30:00.000Z             Air Force   Mountain West   
3564  2018-09-08T23:00:00.000Z  Southern Mississippi  Conference USA   
3565  2018-09-08T23:00:00.000Z  Southern Mississippi  Conference USA   
3566  2018-09-08T23:00:00.000Z  Southern Mississippi  Conference USA   
3567  2018-09-08T23:00:00.000Z  Southern Mississippi  Conference USA   

      home_score         away_team    away_conference  away_score  \
0             19      Rhode Island                CAA          21   
1             35     New Hampshire                CAA           7   
2             66           Shorter         Gulf South           9   
3             42          Davidson            Pioneer          21   
4             49     Robert Morris                NEC          46   
...          ...               ...                ...         ...   
3563          35              Navy  American Athletic           7   
3564          20  Louisiana Monroe           Sun Belt          21   
3565          20  Louisiana Monroe           Sun Belt          21   
3566          20  Louisiana Monroe           Sun Belt          21   
3567          20  Louisiana Monroe           Sun Belt          21   

                                                  lines  
0                                                   NaN  
1                                                   NaN  
2                                                   NaN  
3                                                   NaN  
4                                                   NaN  
...                                                 ...  
3563  {'provider': 'teamrankings', 'spread': 2.5, 'f...  
3564  {'provider': 'Caesars', 'spread': -5.0, 'forma...  
3565  {'provider': 'consensus', 'spread': -5.5, 'for...  
3566  {'provider': 'numberfire', 'spread': -5.5, 'fo...  
3567  {'provider': 'teamrankings', 'spread': -5.0, '...  

[3568 rows x 14 columns]

In [18]:
betting_norm = pd.json_normalize(betting_2018_df['lines'])

In [19]:
betting_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3568 entries, 0 to 3567
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            3568 non-null   int64 
 1   id               3568 non-null   int64 
 2   season           3568 non-null   int64 
 3   week             3568 non-null   int64 
 4   season_type      3568 non-null   object
 5   date             3568 non-null   object
 6   start_date       3568 non-null   object
 7   home_team        3568 non-null   object
 8   home_conference  3567 non-null   object
 9   home_score       3568 non-null   int64 
 10  away_team        3568 non-null   object
 11  away_conference  3554 non-null   object
 12  away_score       3568 non-null   int64 
 13  lines            2887 non-null   object
dtypes: int64(6), object(8)
memory usage: 390.4+ KB


In [20]:
betting_norm

provider  spread           formatted_spread  spread_open  \
0              NaN     NaN                        NaN          NaN   
1              NaN     NaN                        NaN          NaN   
2              NaN     NaN                        NaN          NaN   
3              NaN     NaN                        NaN          NaN   
4              NaN     NaN                        NaN          NaN   
...            ...     ...                        ...          ...   
3563  teamrankings     2.5                  Navy -2.5          NaN   
3564       Caesars    -5.0    Southern Mississippi -5          NaN   
3565     consensus    -5.5  Southern Mississippi -5.5          NaN   
3566    numberfire    -5.5  Southern Mississippi -5.5          NaN   
3567  teamrankings    -5.0    Southern Mississippi -5          NaN   

      over_under  over_under_open  home_moneyline  away_moneyline  
0            NaN              NaN             NaN             NaN  
1            NaN              NaN             NaN             NaN  
2            NaN              NaN             NaN             NaN  
3            NaN              NaN             NaN             NaN  
4            NaN              NaN             NaN             NaN  
...          ...              ...             ...             ...  
3563        47.0              NaN             NaN             NaN  
3564        67.5              NaN             NaN             NaN  
3565        67.5              NaN             NaN             NaN  
3566        67.5              NaN             NaN             NaN  
3567        67.5              NaN             NaN             NaN  

[3568 rows x 8 columns]

In [22]:
lines_2018 = betting_2018_df.merge(betting_norm, left_index = True, right_index=True)

In [23]:
lines_2018 = lines_2018.dropna(subset=['lines'])

In [25]:
lines_2018_con = lines_2018[lines_2018['provider'] == 'consensus']

In [26]:
lines_2018_con.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 769 entries, 44 to 3565
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             769 non-null    int64  
 1   id                769 non-null    int64  
 2   season            769 non-null    int64  
 3   week              769 non-null    int64  
 4   season_type       769 non-null    object 
 5   date              769 non-null    object 
 6   start_date        769 non-null    object 
 7   home_team         769 non-null    object 
 8   home_conference   769 non-null    object 
 9   home_score        769 non-null    int64  
 10  away_team         769 non-null    object 
 11  away_conference   769 non-null    object 
 12  away_score        769 non-null    int64  
 13  lines             769 non-null    object 
 14  provider          769 non-null    object 
 15  spread            768 non-null    float64
 16  formatted_spread  769 non-null    object 


In [27]:
lines_2018_con = lines_2018_con.drop(['index','start_date','lines','provider','spread_open','over_under_open','home_moneyline','away_moneyline','over_under'], axis=1)


In [28]:
# shift column 'Name' to first position 
twelve_column = lines_2018_con.pop('formatted_spread') 
  
# insert column using insert(position,column_name, 
# first_column) function 
lines_2018_con.insert(11, 'formatted_spread', twelve_column) 

In [30]:
lines_2018_con = lines_2018_con.sort_values(by=['date','id'])


In [31]:
lines_2018_con.rename(columns = {'spread':'home_spread'}, inplace = True)

In [32]:
lines_2018_con['home_result'] = lines_2018_con['away_score'] - lines_2018_con['home_score']

In [33]:
lines_2018_con['home_diff'] = lines_2018_con['home_spread'] - lines_2018_con['home_result']

In [34]:
lines_2018_con['away_spread'] = lines_2018_con['home_spread']*-1
lines_2018_con['away_result'] = lines_2018_con['home_result']*-1
lines_2018_con['away_diff'] = lines_2018_con['home_diff']*-1

In [35]:
teams = []
for x in lines_2018_con.iterrows():
    if x[1]['home_diff'] > 0:
        teams.append(x[1]['home_team'])
    else:
        teams.append(x[1]['away_team'])

In [36]:
lines_2018_con['covering_team'] = teams

In [38]:
lines_2018_con['vegas_error'] = abs(lines_2018_con['home_diff'])

In [40]:
lines_2018_con.to_csv('lines_2018.csv')